In [1]:
import torch

## creating tensors

In [8]:
# tensor filled with uninitialized data
em = torch.empty(2)
print(em)

tensor([       nan, 1.8623e-42])


In [9]:
em = torch.empty(1,2)
print(em)

tensor([[0., 0.]])


In [10]:
# Initializing Tensor Directly from data
x = torch.tensor([2.5, .1])
print(x)

tensor([2.5000, 0.1000])


In [11]:
x = torch.rand(5,3)
print(x)
print(x[:, 0])
print(x.size())

tensor([[0.2214, 0.6021, 0.6976],
        [0.8427, 0.5447, 0.6803],
        [0.1191, 0.8889, 0.4112],
        [0.9020, 0.9231, 0.1585],
        [0.0677, 0.9606, 0.7129]])
tensor([0.2214, 0.8427, 0.1191, 0.9020, 0.0677])
torch.Size([5, 3])


## tensor operations

In [12]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[7, 8], [9, 10], [11, 12]])
c = torch.matmul(a, b)
c

tensor([[ 58,  64],
        [139, 154]])

## statistical operation

In [13]:

tensor = torch.Tensor([[1, 2],
                       [3, 4]])

# get minmum value in each row
tensor_min = torch.min(tensor, dim=1)
# get max value in tensor
tensor_max = torch.max(tensor)
# calculate mean value
tensor_mean = torch.mean(tensor)
...  # Every single mathematical function you could imagine.

print(tensor_min, tensor_max, tensor_mean)

torch.return_types.min(
values=tensor([1., 3.]),
indices=tensor([0, 0])) tensor(4.) tensor(2.5000)


In [14]:
tensor = torch.Tensor([[1, 2],
                       [3, 4],
                       [5, 6]])  # shape = (3, 2)

tensor_mean_row = torch.mean(tensor, dim=0)  # shape = (2,) Averaging over 1st dimension (along columns)
tensor_mean_col = torch.mean(tensor, dim=1)  # shape = (3,) Averaging over 2nd dimension (along rows)

print(tensor_mean_row, tensor_mean_col) 

tensor([3., 4.]) tensor([1.5000, 3.5000, 5.5000])


!![image.png](https://i.stack.imgur.com/ORqaP.png)

In [15]:
x = torch.rand(4,4)
print(x)
# view function is a view of the tensor, it does not create a new tensor
# but modifies the original tensor and changes the shape
y = x.view(-1,4 )

z = x.view(1,16 )
print(y.size())
print(z.size())

tensor([[0.1627, 0.1329, 0.0312, 0.3476],
        [0.5599, 0.7680, 0.3398, 0.5478],
        [0.9847, 0.0399, 0.6875, 0.2004],
        [0.6235, 0.6870, 0.6036, 0.2973]])
torch.Size([4, 4])
torch.Size([1, 16])


In [2]:
a = torch.ones(5)
print(a)
b= a.numpy()
print(b)
print(f"type of b {type(b)}")
c = torch.from_numpy(b)
print(c)
print(f"type of c {type(c)}")

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
type of b <class 'numpy.ndarray'>
tensor([1., 1., 1., 1., 1.])
type of c <class 'torch.Tensor'>


In [17]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    x = torch.ones(5, device=device)
    y = torch.ones(5)
    # By default, tensors are created on the CPU. We need to explicitly move tensors 
    # to the GPU using .to method (after checking for GPU availability)
    y = y.to(device)
    z = x*y
    z=z.to("cpu")

## Model Training

```
The general framework for training your model is as follows
for epoch in range(num_epochs):
	for i in range(num_batches):
		Forward pass
		Loss calculation
		Backward pass
		Weights update
```


### Imports

In [18]:
# Combines arrays in a vertically stacked sequence (used for data manipulation)
from numpy import vstack

# Reads a CSV file into a DataFrame (used for loading datasets)
from pandas import read_csv

# Encodes categorical labels into numerical format (used for label preprocessing)
from sklearn.preprocessing import LabelEncoder

# Calculates the accuracy of a classification model (used for model evaluation)
from sklearn.metrics import accuracy_score

# Defines a custom dataset class for PyTorch (used for handling data)
from torch.utils.data import Dataset

# Creates a DataLoader for efficient batch processing in PyTorch (used for data loading)
from torch.utils.data import DataLoader

# Splits a dataset into training and validation sets (used for data splitting)
from torch.utils.data import random_split

# Represents a multi-dimensional matrix in PyTorch (used for tensor manipulation)
from torch import Tensor

# Implements a linear layer in a neural network (used for defining neural network architecture)
from torch.nn import Linear

# Applies rectified linear unit (ReLU) activation function (used for introducing non-linearity)
from torch.nn import ReLU

# Applies sigmoid activation function (used for binary classification output)
from torch.nn import Sigmoid

# Base class for all neural network modules in PyTorch (used for creating custom models)
from torch.nn import Module

# Stochastic Gradient Descent optimizer (used for model optimization during training)
from torch.optim import SGD

# Binary Cross Entropy Loss function (used for binary classification problems)
from torch.nn import BCELoss

# Initializes weights using Kaiming uniform initialization (used for weight initialization)
from torch.nn.init import kaiming_uniform_

# Initializes weights using Xavier (Glorot) uniform initialization (used for weight initialization)
from torch.nn.init import xavier_uniform_


### Data Preparation

PyTorch provides two data primitives: ``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``
that allow you to use pre-loaded datasets as well as your own data.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset`` to enable easy access to the samples.

In [62]:

# dataset definition
# A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.
class CSVDataset(Dataset):
    # load the dataset
    # The __init__ function is run once when instantiating the Dataset object
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        original_labels = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(original_labels)
        
        # this dictionary 
        self.encoding_mapping = {encoded_label: original_label for encoded_label, original_label  in zip(self.y, original_labels)}

        # ensure the target is float
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    # The __len__ function returns the number of samples in our dataset.
    def __len__(self):
        return len(self.X)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])


# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    # The Dataset retrieves our dataset’s features and labels one sample at a time.
    # While training a model, we typically want to pass samples in “minibatches”,
    # reshuffle the data at every epoch to reduce model overfitting,
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return dataset.encoding_mapping, train_dl, test_dl

### Model definition


In [63]:

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        # initializes the weights of the self.hidden1 layer using the Kaiming (He) Uniform initialization method
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        # initializes the weights of the self.hidden3 layer using the Xavier (Glorot) Uniform initialization method.
        #it is commonly used for layers with hyperbolic tangent (tanh) or sigmoid activation functions.
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

### Model training

In [58]:

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss() # Binary Cross-Entropy
    # Stochastic Gradient Descent Optimizer
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

### Model Evaluation

In [59]:

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc


### Model Predictioin

In [60]:


# make a class prediction for one row of data
def predict(row, model: Module):
    # convert row to data
    model.eval() # This is equivalent with self.train(False)
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()

    return yhat

### Model training (main)

In [70]:
# prepare the data
# dataset link https://datahub.io/machine-learning/ionosphere
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
encoding_mapping, train_dl, test_dl = prepare_data(path)
print(train_dl)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(34)

model.train(True) # set it to False for inference 


# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)

print('Predicted: %.3f (class=%d) which is (class=%s)' % (yhat, yhat.round(), encoding_mapping[int(yhat.round())]))

235 116
Accuracy: 0.905
Predicted: 1.000 (class=1) which is (class=g)


# Resources

1. https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
